In [9]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import boto3
import json
import uuid

In [18]:
Target_year = 2023

pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.1f}'.format

User_Name = 'dsadmin'


AWS_folder = "./"

src_folder = os.path.join(AWS_folder,'Downloaded',User_Name, 'geojson')
out_folder = os.path.join(AWS_folder,'ToUpload',User_Name, 'geojson')
info_folder = os.path.join(AWS_folder,'ToUpload',User_Name)

In [19]:
files = glob.glob(os.path.join(info_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]
target_files

['./ToUpload\\dsadmin\\area_def.json', './ToUpload\\dsadmin\\course_info.json']

In [21]:
area_def = json.load(open(target_files[0], encoding="utf8"))

In [22]:
area_def

[{'TypeId': 0,
  'name': 'Undefined',
  'color': '#8d99ae',
  'display': True,
  'DSZindex': 100},
 {'TypeId': 1,
  'name': '전면',
  'color': '#00a0e9',
  'display': False,
  'DSZindex': 1},
 {'TypeId': 2,
  'name': '코스',
  'color': '#B71C1C',
  'display': False,
  'DSZindex': 2},
 {'TypeId': 3,
  'name': '홀영역',
  'color': '#1A237E',
  'display': False,
  'DSZindex': 3},
 {'TypeId': 4,
  'name': '그린',
  'color': '#336600',
  'display': False,
  'DSZindex': 4},
 {'TypeId': 5,
  'name': '티박스',
  'color': '#0D47A1',
  'display': False,
  'DSZindex': 5},
 {'TypeId': 6,
  'name': '훼어웨이',
  'color': '#FFFF33',
  'display': False,
  'DSZindex': 6},
 {'TypeId': 7,
  'name': '기타관심지역',
  'color': '#d90429',
  'display': True,
  'DSZindex': 200},
 {'TypeId': 8,
  'name': '병충해',
  'color': '#d90429',
  'display': True,
  'DSZindex': 101}]

In [11]:
# Create Out folder if not exists
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [36]:

client = boto3.client('s3')

response = client.list_objects_v2(
    Bucket='dsmapdata',
    Prefix=User_Name+'/geojson/')

list = [x['Key'] for x in response.get('Contents', [])]
list = [ x for x in list if ('MGC' in x) ]
list


['dsadmin/geojson/MGC001.json',
 'dsadmin/geojson/MGC002.json',
 'dsadmin/geojson/MGC003.json',
 'dsadmin/geojson/MGC004.json']

base.json file이 모든 정보를 가지고 있으므로 이화일만 다운로드
그 후 내용 화일로 분리

In [37]:
s3 = boto3.client('s3')

for file_ in list:
  content_object = s3.get_object(Bucket = 'dsmapdata', Key = file_)
  file_content = content_object['Body'].read().decode('utf-8')
  json_content = json.loads(file_content)

  for json_ in json_content['features']:
    json_['properties'] = {
            "Name": "Contour_",
            "Id":str(uuid.uuid4()),
            "Client": json_['properties']['Client'],
            "Course": json_['properties']['Course'],            
            "Hole": 0 if json_['properties']['Hole'] == '' else int(float(json_['properties']['Hole'])),
            "Type": json_['properties']['Type'],
            "TypeId": next(item for item in area_def if item['name'] == json_['properties']['Type'])['TypeId'],
            "Desc": ""
          }
  file_to_save = os.path.join(out_folder ,file_.split('/')[2])
  print(file_to_save)
  with open(file_to_save, "w", encoding="utf-8") as file:
    file.write(json.dumps(json_content, ensure_ascii=False))

./ToUpload\dsadmin\geojson\MGC001.json
./ToUpload\dsadmin\geojson\MGC002.json
./ToUpload\dsadmin\geojson\MGC003.json
./ToUpload\dsadmin\geojson\MGC004.json


3

In [38]:
files = glob.glob(os.path.join(out_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]
target_files

['./ToUpload\\dsadmin\\geojson\\MGC001.json',
 './ToUpload\\dsadmin\\geojson\\MGC002.json']

In [39]:
os.path.splitext(os.path.basename(target_files[0]))[0]

'MGC001'

각 화일을 base.json으로 통합, 그후 가가 화일과 통합화일을 Upload

In [40]:
Total_json = {}

s3 = boto3.client('s3')
for _file in target_files:
  f_ = os.path.splitext(os.path.basename(_file))[0]
  json_content_ = json.load(open(_file, encoding="utf8"))
  Total_json[f_] = json_content_
  s3.put_object(Body=json.dumps(json_content_).encode('utf-8'),Bucket='dsmapdata',Key=User_Name+'/geojson/{}.json'.format(f_))
  
# s3.put_object(
#   Body=json.dumps(Total_json).encode('utf-8'),
#   Bucket='dsmapdata',
#   Key=User_Name + '/base.json'
#   )